In [59]:
import os
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
import pathlib
# from tslearn.clustering import TimeSeriesKMeans, silhouette_score

BASE_DATA_PATH = r'C:\D\Whitireia\ARP\data'

In [60]:
def split_df_by_station_id(df: pd.DataFrame) -> list:
    return list(df.groupby('station_id'))

In [61]:
def get_time_period(df: pd.DataFrame, start_date: str, end_date: str) -> pd.DataFrame:
    flt = (df['date'] >= start_date) & (df['date'] <= end_date)
    return df.loc[flt].sort_values(['station_id', 'date'], ascending=[True, False]).reset_index(drop=True)

In [62]:
path_filled_stations = os.path.join(BASE_DATA_PATH, r'stations_filled_all_data')
stations = os.listdir(path_filled_stations)
stations

['12442.csv',
 '18234.csv',
 '21938.csv',
 '25354.csv',
 '25531.csv',
 '2592.csv',
 '2685.csv',
 '3145.csv',
 '31857.csv',
 '3445.csv',
 '40750.csv',
 '40984.csv',
 '41212.csv',
 '41229.csv',
 '41559.csv',
 '8567.csv']

In [67]:
list_all_stations_raw = []
for station in stations:
    df = pd.read_csv(os.path.join(
        path_filled_stations, station), 
        parse_dates=['Observation time UTC']
        ).sort_values('Observation time UTC').reset_index(drop=True)
    
    list_all_stations_raw.append(df)
    # print(type(df['Observation time UTC'][0]))
    # display(df)
display(list_all_stations_raw[15])
print(len(list_all_stations_raw))

,Observation time UTC,station_id,Temperature,Screen_Observations,Rain,Wind
0,1993-03-17,8567.0,15.6,NaN,1.8,NaN
1,1993-03-18,8567.0,12.3,NaN,0.0,4.63
2,1993-03-19,8567.0,12.0,NaN,0.0,4.84
3,1993-03-20,8567.0,14.3,NaN,0.0,3.87
4,1993-03-21,8567.0,12.6,NaN,3.2,7.16
...,...,...,...,...,...,...
11579,2024-12-05,8567.0,NaN,NaN,0.2,4.90
11580,2024-12-06,8567.0,NaN,NaN,0.0,3.15
11581,2024-12-07,8567.0,NaN,NaN,NaN,6.69
11582,2024-12-08,8567.0,NaN,NaN,NaN,7.72


16


In [68]:
df_all_stations_raw = pd.concat(list_all_stations_raw, ignore_index=True)
df_all_stations_raw = df_all_stations_raw.astype({'station_id': int})
df_all_stations_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135991 entries, 0 to 135990
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Observation time UTC  135991 non-null  datetime64[ns]
 1   station_id            135991 non-null  int64         
 2   Temperature           118923 non-null  float64       
 3   Screen_Observations   86687 non-null   float64       
 4   Rain                  127929 non-null  float64       
 5   Wind                  109635 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 6.2 MB


In [69]:
new_columns = ['date', 'station_id', 'temperature', 'rel_humidity', 'rain', 'wind']
df_all_stations_raw.columns = new_columns
df_all_stations_raw

,date,station_id,temperature,rel_humidity,rain,wind
0,1993-03-17,12442,NaN,NaN,NaN,NaN
1,1993-03-18,12442,NaN,NaN,NaN,4.63
2,1993-03-19,12442,NaN,NaN,NaN,4.84
3,1993-03-20,12442,NaN,NaN,NaN,3.87
4,1993-03-21,12442,NaN,NaN,NaN,7.16
...,...,...,...,...,...,...
135986,2024-12-05,8567,NaN,NaN,0.2,4.90
135987,2024-12-06,8567,NaN,NaN,0.0,3.15
135988,2024-12-07,8567,NaN,NaN,NaN,6.69
135989,2024-12-08,8567,NaN,NaN,NaN,7.72


In [70]:
df_all_stations_raw.to_csv(os.path.join(BASE_DATA_PATH, r'df_all_stations_raw.csv'), index=False)

In [ ]:
df_all_stations_raw = pd.read_csv(os.path.join(BASE_DATA_PATH, r'df_all_stations_raw.csv'))
df_all_stations_raw

,date,station_id,temperature,rel_humidity,rain,wind
0,1993-03-17,12442,NaN,NaN,NaN,NaN
1,1993-03-18,12442,NaN,NaN,NaN,4.63
2,1993-03-19,12442,NaN,NaN,NaN,4.84
3,1993-03-20,12442,NaN,NaN,NaN,3.87
4,1993-03-21,12442,NaN,NaN,NaN,7.16
...,...,...,...,...,...,...
135986,2024-12-05,8567,NaN,NaN,0.2,4.90
135987,2024-12-06,8567,NaN,NaN,0.0,3.15
135988,2024-12-07,8567,NaN,NaN,NaN,6.69
135989,2024-12-08,8567,NaN,NaN,NaN,7.72


In [72]:
start_date = '2020-01-11'
end_date = '2023-01-11'

df_all_stations_period = get_time_period(df_all_stations_raw, start_date=start_date, end_date=end_date)
print(df_all_stations_period.info())
df_all_stations_period

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17481 entries, 0 to 17480
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          17481 non-null  object 
 1   station_id    17481 non-null  int64  
 2   temperature   17396 non-null  float64
 3   rel_humidity  17416 non-null  float64
 4   rain          17178 non-null  float64
 5   wind          16353 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 819.5+ KB
None


,date,station_id,temperature,rel_humidity,rain,wind
0,2023-01-11,2592,15.900000,98.041667,34.6,9.65
1,2023-01-10,2592,16.400000,95.041667,67.0,7.85
2,2023-01-09,2592,16.600000,86.250000,0.0,5.43
3,2023-01-08,2592,14.900000,93.291667,3.4,10.52
4,2023-01-07,2592,15.700000,100.000000,59.8,8.31
...,...,...,...,...,...,...
17476,2020-01-15,41559,15.334783,59.217391,0.0,4.33
17477,2020-01-14,41559,14.154167,62.500000,0.0,3.72
17478,2020-01-13,41559,14.458333,69.291667,1.6,4.95
17479,2020-01-12,41559,17.252174,87.173913,11.0,3.96


In [73]:
# rows to 
incomplete_rows = df_all_stations_period[df_all_stations_period.isna().values.any(axis=1)]
display(incomplete_rows)
drop_dates = incomplete_rows['date'].to_list()

# result df without dropped dates
df_all_stations_period_dropped_nan = df_all_stations_period[~df_all_stations_period['date'].isin(drop_dates)]
display(df_all_stations_period_dropped_nan)
print(df_all_stations_period_dropped_nan.info())

,date,station_id,temperature,rel_humidity,rain,wind
52,2022-11-20,2592,15.300000,87.458333,NaN,4.81
54,2022-11-18,2592,15.000000,93.875000,18.0,NaN
56,2022-11-16,2592,18.700000,84.875000,0.0,NaN
58,2022-11-14,2592,13.300000,82.958333,0.0,NaN
59,2022-11-13,2592,17.100000,89.916667,6.2,NaN
...,...,...,...,...,...,...
17368,2020-05-02,41559,17.747619,75.761905,NaN,5.00
17383,2020-04-17,41559,16.136364,92.681818,NaN,2.26
17395,2020-04-05,41559,13.254167,83.916667,0.0,NaN
17402,2020-03-29,41559,13.787500,86.583333,NaN,3.19


,date,station_id,temperature,rel_humidity,rain,wind
12,2022-12-30,2592,18.100000,79.041667,0.6,12.52
18,2022-12-24,2592,17.000000,92.833333,19.4,2.96
19,2022-12-23,2592,17.700000,91.625000,17.2,5.14
121,2022-09-12,2592,14.200000,77.875000,1.4,9.78
124,2022-09-09,2592,11.500000,88.416667,13.8,4.48
...,...,...,...,...,...,...
16665,2022-04-14,41559,10.266667,79.041667,0.0,1.83
16767,2022-01-02,41559,20.841667,71.958333,0.0,1.72
16768,2022-01-01,41559,19.158333,70.375000,0.0,1.77
16774,2021-12-26,41559,19.187500,81.416667,0.0,4.33


<class 'pandas.core.frame.DataFrame'>
Index: 240 entries, 12 to 17474
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          240 non-null    object 
 1   station_id    240 non-null    int64  
 2   temperature   240 non-null    float64
 3   rel_humidity  240 non-null    float64
 4   rain          240 non-null    float64
 5   wind          240 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 13.1+ KB
None


In [21]:
splits = split_df_by_station_id(df_all_stations_period_dropped_nan)

min_len = len(splits[0][1])
min_len_st_id = splits[0][0]
for i,v in enumerate(splits):
    print(i, v[0], len(v[1]))
    if min_len > len(v[1]):
        min_len = len(v[1])
        min_len_st_id = v[0]
print(f"Min rows station: {min_len_st_id}, Min len: {min_len}")

0 2592.0 16
1 2685.0 16
2 3145.0 16
3 3445.0 16
4 8567.0 16
5 12442.0 16
6 18234.0 16
7 21938.0 16
8 25354.0 16
9 25531.0 16
10 40750.0 16
11 40984.0 16
12 41212.0 16
13 41229.0 16
14 41559.0 16
Min rows station: 2592.0, Min len: 16
